In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_excel('datat3.xlsx')
print(data.shape)
data.head()

(922, 33)


,gender,age,SBP,BMI,etiology of CKD,Hb,Alb,Cr,eGFR,CKD_stage,...,eGFR(6M),eGFR(12M),eGFR(18M),eGFR(24M),eGFR(30M),eGFR(36M),eGFR(last visit),average_obs,obsevasion_ duration,fclass
0,2,74,120.0,23.137669,2,12.0,4.0,1.20,89.981926,3,...,26.454698,24.331582,24.682189,21.614854,20.420524,20.420524,18.495328,25.275139,37,0
1,1,57,139.0,28.515625,2,15.9,4.8,0.84,88.330020,2,...,78.287758,71.343858,72.845992,71.908942,71.562914,67.225032,67.225032,72.392152,37,0
2,1,32,154.0,24.582701,4,14.4,4.4,0.87,86.973875,2,...,75.027238,69.595257,68.856399,72.901926,69.749275,69.171408,69.171408,72.694258,36,0
3,1,60,144.0,30.737407,2,14.4,4.7,2.22,86.874201,4,...,26.885061,24.917353,28.581660,29.237135,25.556002,25.183703,25.183703,26.485251,35,0
4,1,49,135.0,NaN,2,17.0,4.1,1.39,86.782629,3,...,46.978867,45.829455,41.488436,41.801561,38.106104,NaN,38.106104,43.081581,30,0


In [13]:
data.columns

Index(['gender', 'age', 'SBP', 'BMI', 'etiology of CKD', 'Hb', 'Alb', 'Cr',
       'eGFR', 'CKD_stage', 'CKD category', 'dip-stick proteinuria',
       'proteinuria', 'urinary occult blood', 'UPCR', 'UPCR category',
       'hypertension', 'prevalence of CVD', 'diabetes', 'use of RAASi',
       'use of CCB', 'use of diuretics', 'eGFR(0M)', 'eGFR(6M)', 'eGFR(12M)',
       'eGFR(18M)', 'eGFR(24M)', 'eGFR(30M)', 'eGFR(36M)', 'eGFR(last visit)',
       'average_obs', 'obsevasion_ duration', 'fclass'],
      dtype='object')

In [12]:
#Fill eGFR
proteinuria2 = data
ff = proteinuria2.columns.get_loc("eGFR_0M")

ll = proteinuria2.columns.get_loc("eGFR_lastvisit")
 
for i in range(len(proteinuria2)):
    xx = proteinuria2.iloc[i, ff:ll+1]
    if xx.isnull().any() == True:
        proteinuria2.iloc[i, ff:ll+1] = xx.interpolate(method ='linear', limit_direction ='backward')



KeyError: 'eGFR_0M'

In [11]:
data['CKD_stage']

0      3
1      2
2      2
3      4
4      3
      ..
917    3
918    4
919    5
920    3
921    4
Name: CKD_stage, Length: 922, dtype: int64

In [7]:
data.isna().sum()

gender                     0
age                        0
SBP                       16
BMI                       98
etiology of CKD            0
Hb                         2
Alb                        9
Cr                         0
eGFR                       0
CKD_stage                  0
CKD category              64
dip-stick proteinuria     10
proteinuria               10
urinary occult blood      10
UPCR                      64
UPCR category             64
hypertension               0
prevalence of CVD          0
diabetes                   0
use of RAASi               1
use of CCB                 0
use of diuretics           0
eGFR(0M)                   0
eGFR(6M)                   0
eGFR(12M)                128
eGFR(18M)                218
eGFR(24M)                268
eGFR(30M)                338
eGFR(36M)                401
eGFR(last visit)           0
average_obs                0
obsevasion_ duration       0
fclass                     0
dtype: int64